In [5]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
import xml.etree.ElementTree as ET
from colorama import Fore, Style

load_dotenv()
client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY'),
)

In [6]:
system_promt = """
Act as an interpreter. Your task is translate given sentence to Chinese (simple chinese) language. Provide the translation only!
Context : ERP system application translation.

Do not translate below words:
GQA, EOL, SOP, PWS, APQP, ...
"""

def translate(text):
    chat_completion = client.chat.completions.create(
        messages = [
            {"role": "system", "content": system_promt},
            {"role": "user", "content": text}
        ],
        model = "gpt-4o",
        temperature = 0.2,
        max_tokens = 3000,
        top_p = 0.1,
        frequency_penalty = 0.2,
        presence_penalty = 0.1,
        stop = None
    )

    response_json = json.loads(chat_completion.model_dump_json(indent = 2))
    content = response_json['choices'][0]['message']['content']
    return content

If you create a new translation please use this cell.

In [ ]:
input_file = 'Input file.g.xlf'
output_file = 'Output file.g.zh-CN.xlf'
language_code = 'zn-CN' #Target language code

count = 0
tree = ET.parse(input_file)
ET.register_namespace('', 'urn:oasis:names:tc:xliff:document:1.2')
root = tree.getroot()
ns = {'xliff': 'urn:oasis:names:tc:xliff:document:1.2'}
trans_unit = root.find('.//xliff:trans-unit', ns)


for trans_unit in root.findall('.//xliff:trans-unit', ns):
    trans_unit_id = trans_unit.attrib.get('id', '')
    if (trans_unit_id.startswith('Page')) or (trans_unit_id.startswith('Report')) or (trans_unit_id.startswith('Codeunit')):
        source = trans_unit.find('xliff:source', ns)
        if source is not None:
            target = trans_unit.find('xliff:target', ns)
            if target is None:
                target = ET.Element('target', attrib={'state': 'translated'})
                trans_unit.append(target)
            
            if source.text:
                target.text = translate(source.text)
                count +=1
                print(f" Count : {count} Forrás : {source.text} -- {Fore.CYAN}Fordítás : {target.text}{Style.RESET_ALL} -- ID : {trans_unit_id}")

file_element = root.find('.//xliff:file', ns)
if file_element is not None:
    file_element.set('target-language', language_code)

tree.write(output_file, encoding='utf-8', xml_declaration=True)
print("Finished.")

If you have an existing translation please use this cell. In this case the existing translations will not send to GPT model.

In [ ]:
input_file = 'Applications.g.xlf'  # Name of system generated xlf, this file created during build.
existing_file = 'Applications.g.hu-HU.xlf'  # Existing translation file.
output_file = 'Applications_translated.xlf'  # New translation file.
language_code = 'hu-HU' # Code of target language.

count = 0
not_found_in_original = False

tree_new = ET.parse(input_file)
tree_existing = ET.parse(existing_file)

ET.register_namespace('', 'urn:oasis:names:tc:xliff:document:1.2')

root_new = tree_new.getroot()
root_existing = tree_existing.getroot()

ns = {'xliff': 'urn:oasis:names:tc:xliff:document:1.2'}

existing_trans_units = {}

for trans_unit in root_existing.findall('.//xliff:trans-unit', ns):
    trans_unit_id = trans_unit.attrib.get('id')
    target = trans_unit.find('xliff:target', ns)
    existing_trans_units[trans_unit_id] = target.text if target is not None else None

for trans_unit in root_new.findall('.//xliff:trans-unit', ns):
    trans_unit_id = trans_unit.attrib.get('id', '')
    if (trans_unit_id.startswith('Page')) or (trans_unit_id.startswith('Report'))  or (trans_unit_id.startswith('Codeunit')):
        source = trans_unit.find('xliff:source', ns)
        if source is not None:
            if trans_unit_id in existing_trans_units:
                existing_target_text = existing_trans_units[trans_unit_id]
                if existing_target_text:
                    target = trans_unit.find('xliff:target', ns)
                    if target is None:
                        target = ET.Element('target', attrib={'state': 'translated'})
                        trans_unit.append(target)
                    
                    target.text = existing_target_text
                    print(f"{Fore.GREEN}ID: {trans_unit_id} átmásolva: {existing_target_text}{Style.RESET_ALL}")
                    continue  
            else:
                not_found_in_original = True

            target = trans_unit.find('xliff:target', ns)
            if target is None:
                target = ET.Element('target', attrib={'state': 'translated'})
                trans_unit.append(target)
            
            if source.text:
                if (not target.text) or (not_found_in_original):
                    target.text = translate(source.text)
                    count += 1
                    print(f" Count: {count} Forrás: {source.text} -- {Fore.CYAN}Fordítás: {target.text}{Style.RESET_ALL} -- ID: {trans_unit_id}")
                    not_found_in_original = False

file_element = root_new.find('.//xliff:file', ns)
if file_element is not None:
    file_element.set('target-language', language_code)

tree_new.write(output_file, encoding='utf-8', xml_declaration=True)

print("Finished.")
